#  Modelo U(1)_BL Dirac Neutrinos

Sample of notebook for specific model

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pandas as pd
import numpy as np
import os, sys, inspect
import commands
import hep as hp
import scipy as sc
import random

from scipy import linalg
import math
from matplotlib.colors import LogNorm

# Neutrino masses and Pontecorvo–Maki–Nakagawa–Sakata matrix

In [3]:
def neutrino_data(IH=True):
    '''From http://www.nu-fit.org/?q=node/166
    and asumming a Normal Hierarchy:
    Output:
    mnu1in: laightest neutrino mass
    Dms2: \Delta m^2_{12}
    Dma2: \Delta m^2_{13}
    ThetSol,ThetAtm,ThetRec: in radians
    '''
    if not IH:
        Dms2=np.array([7.19e-5, 7.39e-5, 7.60e-5])*1e-18 # In GeV
        Dma2=np.array([2.493e-3, 2.523e-3, 2.55e-3])*1e-18 # In GeV
        #input real values:
        #
        ThetSol = np.array([0.298,  0.310,  0.323]) 
        ThetAtm = np.array([0.425,  0.558,  0.578])
        ThetRec = np.array([0.02176, 0.02241, 0.02301])
        DeltaCP = np.array([194*np.pi/180, 222*np.pi/180, 260*np.pi/180])
    else:
        Dms2=np.array([7.19e-5, 7.39e-5, 7.60e-5])*1e-18 # In GeV
        Dma2=np.array([-2.541e-3, -2.509e-3, -2.479e-3])*1e-18 # In GeV
        #input real values:
        #
        ThetSol = np.array([0.298,  0.310,  0.323]) 
        ThetAtm = np.array([0.537,  0.563,  0.582])
        ThetRec = np.array([0.02197, 0.02261, 0.02328])
        DeltaCP = np.array([259*np.pi/180.0, 285*np.pi/180.0, 309*np.pi/180.0])

    return Dms2,Dma2,ThetSol,ThetAtm,ThetRec,DeltaCP

def CasasIbarra(mnu1in=0.0,ranMnu=True,bestfit=True,IH=True):
    #import numpy as np
    #if ranMnu==True:
    #    bestfit=True
        
    Dms2,Dma2,ThetSol,ThetAtm,ThetRec,DeltaCP=neutrino_data(IH)
    
    # Phases of the PMNS matrix
    phases1=np.random.uniform(0.,0.0*np.pi,2)       # cero value for all phases         
    
    delta=1.*(0 if bestfit else np.random.uniform(DeltaCP[0],DeltaCP[2]))
    eta1 =1.*(0 if ranMnu else phases1[1])
    eta2 =1.*(0 if ranMnu else phases1[2])

    if not IH:
        mnu1=1.*(mnu1in      if bestfit else  10**((np.log10(2.5e-3)-np.log10(1e-9))*np.random.uniform(0,1)+np.log10(1e-9))*1e-9) 
        mnu2=1.*(np.sqrt(Dms2[1]+mnu1in**2)     if bestfit else  np.sqrt(np.random.uniform(Dms2[0],Dms2[2]) + mnu1**2)  ) 
        mnu3=1.*(np.sqrt(Dma2[1]+mnu1in**2)     if bestfit else  np.sqrt(np.random.uniform(Dma2[0],Dma2[2]) + mnu1**2)  )

        # Square root of left-handed neutrino mass matrix 
        DMnu = np.diag( [mnu3,mnu2,mnu1] )  
    else:
        mnu3=1.*(mnu1in      if bestfit else  10**((np.log10(2.5e-3)-np.log10(1e-9))*np.random.uniform(0,1)+np.log10(1e-9))*1e-9) 
        mnu2=1.*(np.sqrt(-Dma2[1]+mnu1in**2)     if bestfit else  np.sqrt(np.random.uniform(-Dma2[2],-Dma2[0]) + mnu3**2)  ) 
        mnu1=1.*(np.sqrt(-Dms2[1]+mnu2**2)     if bestfit else  np.sqrt(-np.random.uniform(Dms2[0],Dms2[2]) + mnu2**2)  )
        # Square root of left-handed neutrino mass matrix 
        DMnu = np.diag( [mnu2,mnu1,mnu3] )  

    #print "NEUTRINOS",mnu1,mnu2,mnu3

    # Mixing angles (up 3 sigma range)
    t12 = 1.*( np.arcsin(np.sqrt(ThetSol[1])) if bestfit else np.arcsin(np.sqrt(np.random.uniform(ThetSol[0],ThetSol[2]))))#np.arcsin(np.sqrt(ThetSol[1]))
    t23 = 1.*( np.arcsin(np.sqrt(ThetAtm[1])) if bestfit else np.arcsin(np.sqrt(np.random.uniform(ThetAtm[0],ThetAtm[2]))))
    t13 = 1.*( np.arcsin(np.sqrt(ThetRec[1])) if bestfit else np.arcsin(np.sqrt(np.random.uniform(ThetRec[0],ThetRec[2]))))         #np.arcsin(np.sqrt(ThetRec[1]))

    # Building PMNS matrix
    U12 = np.array([ [np.cos(t12), np.sin(t12),0], [-np.sin(t12), np.cos(t12),0], [0,0,1.0] ])
    U13 = np.array([ [np.cos(t13),0, np.sin(t13)* np.exp(-delta*1j)], [0,1.0,0], [-np.sin(t13)*np.exp(delta*1j),0, np.cos(t13)] ])
    U23 = np.array([ [1.0,0,0], [0, np.cos(t23), np.sin(t23)], [0, -np.sin(t23), np.cos(t23)] ])
    Uphases = np.array([ [np.exp(eta1*1j),0,0], [0, np.exp(eta2*1j),0], [0,0,1.0] ])
    #U = np.dot(U23,np.dot(U13,np.dot(U12,Uphases)))
    U = np.dot(U23,np.dot(U13,U12))
    return U, DMnu


if __name__=='__main__':
    print(np.abs( CasasIbarra(mnu1in=0.0,ranMnu=True,bestfit=True,IH=False) ))

[[[8.21302076e-01 5.50502407e-01 1.49699699e-01]
  [4.63050760e-01 4.89988544e-01 7.38576482e-01]
  [3.33236993e-01 6.75912958e-01 6.57339167e-01]]

 [[5.02294734e-11 0.00000000e+00 0.00000000e+00]
  [0.00000000e+00 8.59651092e-12 0.00000000e+00]
  [0.00000000e+00 0.00000000e+00 0.00000000e+00]]]


# Inverse neutrino mass matrix

In [4]:
def Masses(MH1, MH2, Ms, Theta, Lam5, Lam6, Lam8, Lam7, v, MZp, g1p):
    """ function that transform the coupling at 
        interaction base to physics base 
    """
    vs = MZp/(3.0*g1p)
    v2 = v*v; vs2 = vs*vs; MH12 = MH1*MH1; MH22 = MH2*MH2;
    Mu3 = (MH22-MH12)*np.sin(2.0*Theta)/(np.sqrt(2.0)*vs)
    Mu32 = Mu3*Mu3
    MS22 = 0.5*(MH12+MH22+np.sqrt((MH22-MH12)**2-2.0*Mu32*vs2))
    MS12 = 0.5*(MH12+MH22-np.sqrt((MH22-MH12)**2-2.0*Mu32*vs2))

    Mu12 = MS12+0.5*Lam5*v2+0.5*Lam7*vs2
    Mu22 = MS22+0.5*Lam6*v2+0.5*Lam8*vs2
    L_s = -0.5*(Ms/(vs))**2
    return L_s, Mu3, Mu12, Mu22, vs

def Const(Theta):
    MH12 = MH1*MH1; MH22 = MH2*MH2;
    return np.sin(2.0*Theta)/(16.0*np.pi**2)

def I(M1, M2, m):
    M12 = M1*M1; M22 = M2*M2; m2 = m*m
    return (m*(M22*np.log(M22/m2)/(M22-m2)-M12*np.log(M12/m2)/(M12-m2)))

def Inverse_Matrix(f23 = 1.0E-04, h11 = 1.0E-04, h22 = 1.0E-04, C = 0.001, MH1 = 500, MH2 = 700, IH=False):
    """Inverse neutrino mass matrix normalized by C from
        Mnu_diag=UR^T.(C f.ml.h).UL
        ,
        (Inverse_Zee_Matrix) = UR.Mnu_diag
        sgnm2=-1 guarantees real Yukawa couplings
        
    requires hep.py:
       https://github.com/restrepo/BSM-Toolbox/blob/master/tests/hep.py
    """
    m_e = 0.5109989461e-3; m_m = 0.1056583745; m_t = 1.77686
    ml = [I(MH1, MH2, m_e),I(MH1, MH2, m_m),I(MH1, MH2, m_t)]

    U_MNS, m = CasasIbarra(mnu1in=0.0,ranMnu=True,bestfit=True,IH=IH)
    X = np.dot(U_MNS,m)

    f = np.zeros((3,3))
    h = np.zeros((3,3))
    
    f[1,2] = f23
    h[0,0] = h11
    h[1,1] = h22
            
    f[0,2] = (f[1,2]*(X[0,1]*X[2,0]-X[0,0]*X[2,1]))/(X[1,1]*X[2,0]-X[1,0]*X[2,1])
    f[0,1] = (f[1,2]*(-(X[0,1]*X[1,0])+X[0,0]*X[1,1]))/(-(X[1,1]*X[2,0])+X[1,0]*X[2,1])
    
    f[2,0] =-f[0,2]
    f[2,1] =-f[1,2]
    f[1,0] =-f[0,1]
    
    h[1,0] = (C*f[1,2]*h[0,0]*ml[0]*(-(X[0,1]*X[2,0])+X[0,0]*X[2,1])+X[2,0]*(X[1,1]*X[2,0]-X[1,0]*X[2,1]))/(C*f[1,2]*ml[1]*(X[1,1]*X[2,0]-X[1,0]*X[2,1]))
    h[0,1] = ((C*f[1,2]*h[1,1]*ml[1]-X[2,1])*(-(X[1,1]*X[2,0])+X[1,0]*X[2,1]))/(C*f[1,2]*ml[0]*(X[0,1]*X[2,0]-X[0,0]*X[2,1]))
    h[2,1] = (C*f[1,2]*h[1,1]*ml[1]*(-(X[0,1]*X[1,0])+X[0,0]*X[1,1])+X[0,1]*(-(X[1,1]*X[2,0])+X[1,0]*X[2,1]))/(C*f[1,2]*ml[2]*(X[0,1]*X[2,0]-X[0,0]*X[2,1]))
    h[2,0] = (-(X[1,0]/(C*f[1,2]))+(h[0,0]*ml[0]*(-(X[0,1]*X[1,0])+X[0,0]*X[1,1]))/(-(X[1,1]*X[2,0])+X[1,0]*X[2,1]))/ml[2]
    
    return f, h

In [5]:
a=hp.hep(MODEL='U1BLDirac')

In [6]:
#Todo en Gev
Ms =  300.0
MH1 = 500.0
MH2 = 700.0
MZp = 2500.0
Theta = 0.1
g1p = 0.5
g1p1 = 0.0
g11p = 0.0

LamH = -1.27000000E-01
Lam1 = 0.0000000E-03
Lam2 = 0.0000000E-03
Lam3 = 0.0000000E-03
Lam4 = 0.0000000E-03
Lam5 = 1.0000000E-03
Lam6 = 1.0000000E-03
Lam7 = 1.0000000E-03
Lam8 = 1.0000000E-03
v = 2.46875795E+02

LamS, Mu3, Mu12, Mu22, vs = Masses(MH1, MH2, Ms, Theta, Lam5, Lam6, Lam8, Lam7, v, MZp, g1p)

devnull=commands.getoutput('rm -f SPheno.spc.%s' %a.MODEL)

a.LHA.blocks['SPHENOINPUT'].entries[55]='0               # Calculate one loop masses'
a.LHA.blocks['MINPAR'][1]='%0.8E       #lambdaHInput' %LamH
a.LHA.blocks['MINPAR'][2]='%0.8E       #lambdaSInput' %LamS
a.LHA.blocks['MINPAR'][3]='%0.8E       #lambda1Input' %Lam1
a.LHA.blocks['MINPAR'][4]='%0.8E       #lambda2Input' %Lam2
a.LHA.blocks['MINPAR'][5]='%0.8E       #lambda3Input' %Lam3
a.LHA.blocks['MINPAR'][6]='%0.8E       #lambda4Input' %Lam4
a.LHA.blocks['MINPAR'][7]='%0.8E       #lambda5Input' %Lam5
a.LHA.blocks['MINPAR'][8]='%0.8E       #lambda6Input' %Lam6
a.LHA.blocks['MINPAR'][9]='%0.8E       #lambda7Input' %Lam7
a.LHA.blocks['MINPAR'][10]='%0.8E      #lambda8Input' %Lam8
a.LHA.blocks['MINPAR'][11]='%0.8E      #mu1INPUT'     %Mu12
a.LHA.blocks['MINPAR'][12]='%0.8E      #mu2INPUT'     %Mu22
a.LHA.blocks['MINPAR'][13]='%0.8E      #mu3INPUT'     %Mu3
a.LHA.blocks['MINPAR'][20]='%0.8E      #g1pINPUT'     %g1p
a.LHA.blocks['MINPAR'][21]='%0.8E      #g1p1INPUT'    %g1p1
a.LHA.blocks['MINPAR'][22]='%0.8E      #g11pINPUT'    %g11p
a.LHA.blocks['MINPAR'][30]='%0.8E      #vXinput'      %vs


moc=a.runSPheno()

In [7]:
def sign():
    if np.random.rand()<0.5:
        return +1
    else: return -1

In [8]:
IH = True
df=pd.DataFrame()
npoints=100
hmin=1E-6;hmax=3
hmin1=1E-2;hmax1=3

for i in range(npoints):
    M2 = random.uniform(80, 200)
    M1 = random.uniform(80, M2)

    Theta = random.uniform(0, np.pi/2)
    O=pd.Series({'Theta':Theta,'M1':M1,'M2':M2})
    #for ij in [(1,2),(1,3),(2,1)]:
    #    O[ij]=np.exp(np.random.uniform(np.log(hmin),np.log(hmax)))
    f23=sign()*np.exp(np.random.uniform(np.log(hmin),np.log(hmax)))
    
    h11=sign()*np.exp(np.random.uniform(np.log(hmin),np.log(hmax)))
    h22=sign()*np.exp(np.random.uniform(np.log(hmin),np.log(hmax)))
                
    C = Const(O.Theta)
    O['C']=C
    f, h = Inverse_Matrix(f23 = f23, h11 = h11, h22 = h22, C = C, MH1 = M1, MH2 = M2,IH=IH)
    if np.abs(np.append(h,f)).max()<=hmax and np.abs(np.append(h,f)).max()>=hmin:
        O=O.append(pd.Series({'h11':h[0,0],'h12':h[0,1],'h13':h[0,2],'h21':h[1,0],'h22':h[1,1],'h23':h[1,2],\
                              'h31':h[2,0],'h32':h[2,1],'h33':h[2,2],'f12':f[0,1],'f13':f[0,2],'f23':f[1,2],'h':h,'f':f}))
        df=df.append(O,ignore_index=True)
    
df.shape

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:45: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:46: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:52: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:53: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:54: ComplexWarning: Casting complex values to real discards the imaginary part
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:55: ComplexWarning: Casting complex values to real discards the imaginary part


(60, 18)

# check points

In [9]:
def Mass_neu(f, h, MH1, MH2, Theta):
    m_e = 0.5109989461e-3; m_m = 0.1056583745; m_t = 1.77686

    ml = np.diag( [I(MH1, MH2, m_e),I(MH1, MH2, m_m),I(MH1, MH2, m_t)] )  
    Mv = Const(Theta)*np.dot(f.transpose(),np.dot(ml,h))
    
    A = np.dot(Mv.transpose(),Mv)
    B = np.linalg.eig(A)
    
    A = np.dot(Mv,Mv.transpose())
    C = np.linalg.eig(A)
    
    return np.sqrt(np.abs(B[0])), B[1], np.sqrt(np.abs(C[0])), C[1]

In [42]:
i = 5
Mass_neu(df.f[i], df.h[i], df.M1[i], df.M2[i], df.Theta[i])

(array([5.00899191e-11, 4.93467324e-11, 0.00000000e+00]),
 array([[1.00000000e+00, 9.92934053e-16, 0.00000000e+00],
        [0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 array([5.00899191e-11, 4.33680869e-19, 4.93467324e-11]),
 array([[-0.82121806,  0.15036622, -0.55044609],
        [ 0.46178179,  0.74180224, -0.4862993 ],
        [-0.33519916,  0.65354375,  0.67861778]]))

In [43]:
def Check(M1, M2, M3, IH = False):
    
    if not IH:
        Dms2=np.array([6.79e-5, 8.01e-5])*1e-18 # In GeV
        Dma2=np.array([2.432e-3, 2.618e-3])*1e-18 # In GeV
    
        mnu2_min=np.sqrt(Dms2[0]+M1**2)
        mnu2_max=np.sqrt(Dms2[1]+M1**2)
        mnu3_min=np.sqrt(Dma2[0]+M1**2)
        mnu3_max=np.sqrt(Dma2[1]+M1**2)

        if  M3 >= np.abs(mnu3_min) and M3 <= np.abs(mnu3_max):
            if M2 >= np.abs(mnu2_min) and M2 <= np.abs(mnu2_max):
                if M1 < 1.0E-15 or math.isnan(M1):
                    return True
    else:
        Dms2=np.array([6.79e-5, 8.01e-5])*1e-18 # In GeV
        Dma2=np.array([2.416e-3, 2.603e-3])*1e-18 # In GeV
        
        mnu2_min=np.sqrt(Dma2[0]+M1**2)
        mnu2_max=np.sqrt(Dma2[1]+M1**2)
        mnu1_max=np.sqrt(-Dms2[0]+mnu2_max**2)
        mnu1_min=np.sqrt(-Dms2[1]+mnu2_min**2)
        
        if  M3 >= np.abs(mnu2_min) and M3 <= np.abs(mnu2_max):
            if M2 >= np.abs(mnu1_min) and M2 <= np.abs(mnu1_max):
                if M1 < 1.0E-15 or math.isnan(M1):
                    return True
        
    return False

def Check_Ang(t12, t23, t13): #, ReU, ImU
    t12E=np.array([31.42, 36.05])
    t23E=np.array([0, 360])#np.array([40.3, 51.5])
    t13E=np.array([8.09, 8.98])    
    if  t12 >= t12E[0] and t12 <= t12E[1]:
        if t23 >= t23E[0] and t23 <= t23E[1]:
            if t13 >= t13E[0] and t13 <= t13E[1]:
                return True
    return False

def Check_LFV(FLFV):
    muegamma = 4.2E-13; tauegamma = 3.3E-08; taumugamma = 4.4E-08; mu3e = 1.0E-12; tau3mu = 2.1E-08 
    tauemumu = 2.7E-08; taumuee = 1.8E-08; tau3e = 2.7E-08; mueTi = 4.3E-12; mueAu = 7.0E-13
    LFV = np.array([muegamma, tauegamma, taumugamma, mu3e, tau3mu, tauemumu, taumuee, tau3e, mueTi, mueAu])        
    return np.all(FLFV < LFV)

# Run Sarah

In [51]:
N = df.f12.shape[0]
j = 0
k = 0
df1=pd.DataFrame()

for i in range(N):  
    h11 = df.h[i][0,0]          # h(1,1)
    h12 = df.h[i][0,1]          # h(1,2)
    h21 = df.h[i][1,0]          # h(2,1)
    h22 = df.h[i][1,1]          # h(2,2)
    h31 = df.h[i][2,0]          # h(3,1)
    h32 = df.h[i][2,1]          # h(3,2)
    
    f11 = df.f[i][0,0]          # f(1,1)
    f12 = df.f[i][0,1]          # f(1,2)
    f13 = df.f[i][0,2]          # f(1,3)
    f21 = df.f[i][1,0]          # f(2,1)
    f22 = df.f[i][1,1]          # f(2,2)
    f23 = df.f[i][1,2]          # f(2,3)
    f31 = df.f[i][2,0]          # f(3,1)
    f32 = df.f[i][2,1]          # f(3,2)
    f33 = df.f[i][2,2]          # f(3,3)
    
    a.LHA.blocks['HLIN'][(1,1)]='%0.8E      # hl(1,1)'    %h11
    a.LHA.blocks['HLIN'][(1,2)]='%0.8E      # hl(1,2)'    %h12
    a.LHA.blocks['HLIN'][(2,1)]='%0.8E      # hl(2,1)'    %h21
    a.LHA.blocks['HLIN'][(2,2)]='%0.8E      # hl(2,2)'    %h22
    a.LHA.blocks['HLIN'][(3,1)]='%0.8E      # hl(3,1)'    %h31
    a.LHA.blocks['HLIN'][(3,2)]='%0.8E      # hl(3,2)'    %h32

    a.LHA.blocks['FLIN'][(1,1)]='%0.8E      # fl(1,1)'    %f11
    a.LHA.blocks['FLIN'][(1,2)]='%0.8E      # fl(1,2)'    %f12
    a.LHA.blocks['FLIN'][(1,3)]='%0.8E      # fl(1,3)'    %f13
    a.LHA.blocks['FLIN'][(2,1)]='%0.8E      # fl(2,1)'    %f21
    a.LHA.blocks['FLIN'][(2,2)]='%0.8E      # fl(2,2)'    %f22
    a.LHA.blocks['FLIN'][(2,3)]='%0.8E      # fl(2,3)'    %f23
    a.LHA.blocks['FLIN'][(3,1)]='%0.8E      # fl(3,1)'    %f31
    a.LHA.blocks['FLIN'][(3,2)]='%0.8E      # fl(3,2)'    %f32
    a.LHA.blocks['FLIN'][(3,3)]='%0.8E      # fl(3,3)'    %f33
    
    L_s, Mu3, Mu12, Mu22, vs = Masses(df.M1[i], df.M2[i], Ms, df.Theta[i], Lam5, Lam6, Lam8, Lam7, v, MZp, g1p)
    
    a.LHA.blocks['MINPAR'][2]='%0.8E       #lambdaSInput' %L_s
    a.LHA.blocks['MINPAR'][11]='%0.8E      #mu1INPUT'     %Mu12
    a.LHA.blocks['MINPAR'][12]='%0.8E      #mu2INPUT'     %Mu22
    a.LHA.blocks['MINPAR'][13]='%0.8E      #mu3INPUT'     %Mu3
    a.LHA.blocks['MINPAR'][30]='%0.8E      #vXinput'      %vs

    moc=a.runSPheno()
    
    M1 = a.LHA_out.blocks['MASS'].entries[14]
    M2 = a.LHA_out.blocks['MASS'].entries[8810012]
    M3 = a.LHA_out.blocks['MASS'].entries[8810016]
    
    Mh1 = a.LHA_out.blocks['MASS'].entries[900037]
    Mh2 = a.LHA_out.blocks['MASS'].entries[900038]
    
    sint = a.LHA_out.blocks['CHARGEMIX'].entries[3,2]
    cost = a.LHA_out.blocks['CHARGEMIX'].entries[3,3]
    
    phi = np.arctan2(sint,cost)

    Brmuegamma = a.LHA_out.blocks['FLAVORKITLFV'][701]     #BR(mu->e gamma)
    Brtauegamma = a.LHA_out.blocks['FLAVORKITLFV'][702]    #BR(tau->e gamma)
    Brtaumugamma = a.LHA_out.blocks['FLAVORKITLFV'][703]   #BR(tau->mu gamma)     
    Brmu3e = a.LHA_out.blocks['FLAVORKITLFV'][901]         #BR(mu->3e)
    Brtau3mu = a.LHA_out.blocks['FLAVORKITLFV'][903]       #BR(tau->3mu)    
    Brtauemumu = a.LHA_out.blocks['FLAVORKITLFV'][904]     #BR(tau- -> e- mu+ mu-) 
    Brtaumuee = a.LHA_out.blocks['FLAVORKITLFV'][905]      #BR(tau- -> mu- e+ e-)
    Brtau3e = a.LHA_out.blocks['FLAVORKITLFV'][902]        #BR(tau->3e)
    CrmueTi = a.LHA_out.blocks['FLAVORKITLFV'][801]        #CR(mu-e, Ti) 
    CrmueAu = a.LHA_out.blocks['FLAVORKITLFV'][804]        #CR(mu-e, Au)    
    
    FLFV = np.array([Brmuegamma, Brtauegamma, Brtaumugamma, Brmu3e, Brtau3mu, Brtauemumu, Brtaumuee, Brtau3e, CrmueTi, CrmueAu])    
    
    if Check_LFV(FLFV):
        if Check(np.abs(M1), np.abs(M2), np.abs(M3), IH = IH):
            a.branchings(a.LHA_out.decays,min_pdg=25)
            a.Series=a.Series.append(a.Br_names)        
            a.Series=a.Series.append(hp.block_to_series(a.LHA_out_with_comments.blocks['FLAVORKITLFV']))
            a.Series=a.Series.append(pd.Series({(3,1):df.h[i][2,0],(3,2):df.h[i][2,1],'f23':df.f[i][1,2],'Theta':df.Theta[i],'phi':phi,'Mh1':Mh1,'Mh2':Mh2}))
            
            df1=df1.append(a.Series,ignore_index=True)
            df1=df1.fillna(0)
            
    if (i%(int(N/10))) == 0:
        print(np.round((float(i/(1.0*N))),2), i)
        
df1.shape

(0.0, 0)
(0.1, 6)
(0.2, 12)
(0.3, 18)
(0.4, 24)
(0.5, 30)
(0.6, 36)
(0.7, 42)
(0.8, 48)
(0.9, 54)


(11, 131)